In [123]:
#Import Libraries. 
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import os

In [124]:
#Google Sheet of Spectral Observations
#
#Save as Excel
#Read Excel as Pandas DataFrame, skip top 5 rows
#Fix pathways
Bishop_Spect_gsheet=pd.read_excel('/Users/JR/Google Drive/Skok_ToDo/BishopWork/Bishop_spectral_file_record_20200812_current.xlsx', skiprows=5)
#Clean up Column names
Bishop_Spect_gsheet.columns=['Listed on DMP Site', 'check', 'Passed Quality Control','Revised','SETI','PDS','ID','File Name','Additional','Date','Begin', 'End','Sample Description','Particle Size','Min. Particle', 'Max. Particle','Reference' ]
#Remove lines with no data
Bishop_Spect_gsheet=Bishop_Spect_gsheet[Bishop_Spect_gsheet['File Name'].notna()]
#Create New Columns
Bishop_Spect_gsheet['Sample on Web']=''
Bishop_Spect_gsheet['Description on Web']=''
#Show first line for confirmation
Bishop_Spect_gsheet.head(1)

,Listed on DMP Site,check,Passed Quality Control,Revised,SETI,PDS,ID,File Name,Additional,Date,Begin,End,Sample Description,Particle Size,Min. Particle,Max. Particle,Reference,Sample on Web,Description on Web
0,x,NaN,x,x,x,x,JB-0001,bkr1jb0001,492f01,"July 28, 1992",0.3,25.92,"cation-exchanged montmorillonite, 100% FeCl2 s...",dry sieved <45 µm,0,45,"Bishop, J. L., Pieters, C. M., Edwards, J. O.,...",,


In [125]:
#Read in Spectral Website. 
#Set header to avoid 410 error
headers = {'User-Agent': 'whatever'}
#Enter Website with Spectral Table
response = requests.get('https://dmp.seti.org/jbishop/spectral-library.html#Bishop_et_al___1994_', headers=headers) 
#Read HTML from website
txt=response.text
#Parse HTML to make useful
htmlparse = BeautifulSoup(txt, 'html.parser')
#Extract spectral table
table = htmlparse.find_all('table')
#Read table into a Pandas Datafram
BishopWeb = pd.read_html(str(table))[0]
#Drop NAN values from DataFrame
BishopWeb.dropna(axis = 0, how = 'all', inplace = True)
BishopWeb=BishopWeb[BishopWeb['File Name'].notna()]
#Create New Columns
BishopWeb['Sample On Sheet']=''
BishopWeb['Spectra Link']=''
BishopWeb['Spectra Link Length']=''
BishopWeb['Description on Sheet']=''
#Show first line for confirmation
BishopWeb.head(1)

,Sample ID,File Name,WL Begin (Âµm),WL End (Âµm),File Date,Sample Description; Texture/Particle Size,Reference,Sample On Sheet,Spectra Link,Spectra Link Length,Description on Sheet
0,JB0001,bkr1jb0001,0.3,25.0,28-Jul-1992,"cation-exchanged montmorillonite, 100% FeCl2 s...",Bishop et al. (1994),,,,


In [126]:
#Check Web for Sheet Sample Data
#This code kicks up some caveat warnings but still should work
#Check each row of the Shreadsheet
for index, row in Bishop_Spect_gsheet.iterrows():
    #Check that the file Spreadsheet sample name is on the web 
    ValueName=Bishop_Spect_gsheet['File Name'][index] in BishopWeb['File Name'].values
    #Report True or False on column 
    Bishop_Spect_gsheet['Sample on Web'][index]=ValueName
    #Check that the file Spreadsheet sample description is on the web 
    ValueDes=Bishop_Spect_gsheet['Sample Description'][index] in BishopWeb['Sample Description; Texture/Particle Size'].values
    #Report True or False on column 
    Bishop_Spect_gsheet['Description on Web'][index]=ValueDes

#Show first line for confirmation
Bishop_Spect_gsheet.head(1)

/Users/JR/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/JR/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Listed on DMP Site,check,Passed Quality Control,Revised,SETI,PDS,ID,File Name,Additional,Date,Begin,End,Sample Description,Particle Size,Min. Particle,Max. Particle,Reference,Sample on Web,Description on Web
0,x,NaN,x,x,x,x,JB-0001,bkr1jb0001,492f01,"July 28, 1992",0.3,25.92,"cation-exchanged montmorillonite, 100% FeCl2 s...",dry sieved <45 µm,0,45,"Bishop, J. L., Pieters, C. M., Edwards, J. O.,...",True,False


In [127]:
#Check Sheet for Web Sample
#This code kicks up some caveat warnings but still should work
#Check each row of the Web table
for index, row in BishopWeb.iterrows():
    #Check that the Web sample name is on the Spreadsheet 
    ValueName=BishopWeb['File Name'][index] in Bishop_Spect_gsheet['File Name'].values
    #Report True or False on column 
    BishopWeb['Sample On Sheet'][index]=ValueName
    #Check that the Web sample description is on the Spreadsheet
    ValueDes=BishopWeb['Sample Description; Texture/Particle Size'][index] in Bishop_Spect_gsheet['Sample Description'].values
    #Report True or False on column 
    BishopWeb['Description on Sheet'][index]=ValueDes

#Show first line for confirmation
BishopWeb.head(1)

/Users/JR/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/JR/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Sample ID,File Name,WL Begin (Âµm),WL End (Âµm),File Date,Sample Description; Texture/Particle Size,Reference,Sample On Sheet,Spectra Link,Spectra Link Length,Description on Sheet
0,JB0001,bkr1jb0001,0.3,25.0,28-Jul-1992,"cation-exchanged montmorillonite, 100% FeCl2 s...",Bishop et al. (1994),True,,,False


In [128]:
#Check for Broken Spectral Txt links
#This script searches for https links in the table to extract links to txt file. 
#It then opens the txt link and calculates the length. A working spectral file is >40000 characters. 
#A file not found or 'BAD link is about 325 characters long. 
#Known bug: In same (~25 during the first run) cases where the spreadsheet sample is not on the web, this code sometimes doesn't write the file length or link quality.
#It is able to open and measure is but for an unknown reason, doesn't write to WebCheck file. These were manually checked and filled in by hand. 


#Set header to avoid 410 error
headers = {'User-Agent': 'whatever'}
#Find links, where html are tagged with an 'a'
links = htmlparse.find_all('a')
#Go through all links
for tag in links:
    #Convert link url to text string  
    tag_string=str(tag)
    #Check if link is for spectral file, which all have 'https'
    if 'https' in tag_string:
        #extract url from html tag
        url=tag.get('href')
        #print(url)
        name=url
        #Get name of spectra from url
        name=os.path.basename(name)
        name=os.path.splitext(name)[0]
        namel=name.lower()
        #print(namel)
        #Get file from url
        response= requests.get(url, headers=headers)
        #convert file to text
        txt=response.text
        #Print length of text
        #print(len(txt))
        #Write length of spectra text file
        BishopWeb.loc[BishopWeb['File Name'] == namel, ['Spectra Link Length']]=str(len(txt))
        #Write if link is Good or Bad
        if len(txt) < 500:
            BishopWeb.loc[BishopWeb['File Name'] == namel, ['Spectra Link']]='Bad'
            #print('BAD')
        else:
            BishopWeb.loc[BishopWeb['File Name'] == namel, ['Spectra Link']]='Good'
            #print('GOOD')

#Show first line for confirmation            
BishopWeb.head(1)

,Sample ID,File Name,WL Begin (Âµm),WL End (Âµm),File Date,Sample Description; Texture/Particle Size,Reference,Sample On Sheet,Spectra Link,Spectra Link Length,Description on Sheet
0,JB0001,bkr1jb0001,0.3,25.0,28-Jul-1992,"cation-exchanged montmorillonite, 100% FeCl2 s...",Bishop et al. (1994),True,Good,55293,False
2,JB0002,bkr1jb0002,0.3,25.0,28-Jul-1992,"cation-exchanged montmorillonite, 200% FeCl2 s...",Bishop et al. (1994),True,Good,51819,False
4,JB0004,bkr1jb0004,0.3,25.0,28-Jul-1992,"cation-exchanged montmorillonite, 600% FeCl2 s...",Bishop et al. (1994),True,Good,51818,False
5,JB0005,bkr1jb0005,0.3,25.0,28-Jul-1992,"cation-exchanged montmorillonite, 100% HCl sol...",Bishop et al. (1994),True,Good,47457,False
6,JB0006,bkr1jb0006,0.3,25.0,10-Jul-1992,"cation-exchanged montmorillonite, 100% CaCl2 s...",Bishop et al. (1994),True,Good,47518,False


In [ ]:
#Confirm that Columns are filled
BishopWeb.head(1)

In [ ]:
#Confirm that Columns are filled
Bishop_Spect_gsheet.head(1)

In [117]:
#Save to excel
#Fix pathways
BishopWeb.to_excel("/Users/JR/Google Drive/Skok_ToDo/BishopWork/WebCheck_8_22_20.xlsx")  
Bishop_Spect_gsheet.to_excel("/Users/JR/Google Drive/Skok_ToDo/BishopWork/SheetCheck_8_22_20.xlsx")  